In [2]:
import pysam
from tqdm import tqdm
from itertools import islice
import pandas as pd
from IPython.display import display, HTML

```
$ conda run bwa index MG1655-K12.first400K.fasta
$ conda run bwa mem data/MG1655-K12.first400K.fasta (zcat ./data/ecoli_400K_err_1.fastq.gz |
psub) (zcat ./data/ecoli_400K_err_2.fastq.gz | psub) > ecoli.sam
$ conda run samtools sort -o ecoli.bam ecoli.sam
```

### Quake
 
 не работает и не чинится
 
```
Guessing quality values are on ascii 33 scale
Processing sequences...
..2763204 sequences processed, 276320400 bp scanned
WARNING: Input had 560786 non-DNA (ACGT) characters who
7167988 total distinct mers
7167988 mers occur at least 0 times
Traceback (most recent call last):
  File "../data/Quake/bin/quake.py", line 134, in <modu
    main()
  File "../data/Quake/bin/quake.py", line 72, in main
    cutoff = open('cutoff.txt').readline().rstrip()
IOError: [Errno 2] No such file or directory: 'cutoff.t
conda 4.6.14

```

### Spades

```
$ conda run bwa mem data/MG1655-K12.first400K.fasta (zcat ./spades/corrected/ecoli_400K_err_1.00.0_0.cor.fastq.gz | psub) (zcat ./spades/corrected/ecoli_400K_err_2.00.0_0.cor.fastq.gz  | ps
ub) > ecoli_spades.sam
$ conda run samtools sort -o ecoli_spades.bam ecoli_spades.sam
```

### Идея алгоритма

Делаем pileup (выстраеваем риды под референсом и смотрим по колонкам), считаем, для каждого нуклеотида из референса, какие риды до/после Спэйдса совпали с референсом/не совпали/отсутствуют. Пересекаем множества.

In [6]:
table = pd.DataFrame.from_dict(
                     {'Error in raw data' : [0, 0, 0],
                      'Correct base in raw data' : [0, 0, 0],
                     }, orient='index', 
                        columns=['Error in corrected reads', 'Correct base in corrected reads', 'Base is absent in corrected reads']
)

reference = pysam.FastaFile("work/data/MG1655-K12.first400K.fasta")
with pysam.AlignmentFile('work/ecoli.bam', 'rb') as regfile:
    with pysam.AlignmentFile('work/ecoli_spades.bam', 'rb') as corrfile:
        #for regcol, corrcol in tqdm(islice(zip(regfile.pileup(fastafile=reference), corrfile.pileup(fastafile=reference)),100_00)):
        for regcol, corrcol in tqdm(zip(regfile.pileup(fastafile=reference), corrfile.pileup(fastafile=reference))):
            try:
                corr = {'err':set(), 'match':set(), 'absent':set()}
                reg = {'err':set(), 'match':set()}

                corrseq = corrcol.get_query_sequences(mark_matches=True)
                corrnames = corrcol.get_query_names()
                corrzip = zip(corrseq, corrnames)

                regseq = regcol.get_query_sequences(mark_matches=True)
                regnames = regcol.get_query_names()
                regzip = zip(regseq, regnames)

                for t, n in corrzip:
                    if t in ",.":
                        corr['match'].add(n)
                    elif t in "*<>N":
                        corr['absent'].add(n)
                    else:
                        corr['err'].add(n)


                for t, n in regzip:
                    if t in ",.":
                        reg['match'].add(n)
                    else:
                        reg['err'].add(n)

                table.iloc[0,0] += len(reg['err']   & corr['err'])
                table.iloc[1,0] += len(reg['match'] & corr['err'])
                table.iloc[0,1] += len(reg['err']   & corr['match'])
                table.iloc[1,1] += len(reg['match'] & corr['match'])
                table.iloc[0,2] += len(reg['err']   & (set(regseq) - corr['match'] - corr['err']))
                table.iloc[1,2] += len(reg['match'] & (set(regseq) - corr['match'] - corr['err']))
            except:
                continue
display(HTML(table.to_html()))

400000it [35:47, 186.24it/s]


,Error in corrected reads,Correct base in corrected reads,Base is absent in corrected reads
Error in raw data,120731,261277,0
Correct base in raw data,255,239158504,0
